In [1]:
import pandas as pd
import numpy as np
#from matplotlib import pyplot as plt
import sklearn
import scipy.stats as stats
import re as re

# loading the dataset
train_titanic = pd.read_csv(r'C:\Users\msrg\Desktop\titanic\train.csv', header = 0, dtype={'Age': np.float64})
train_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
test_titanic = pd.read_csv(r'C:\Users\msrg\Desktop\titanic\test.csv', header = 0, dtype={'Age': np.float64})
test_titanic.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [3]:
# results set that needs to be produced
result_titanic = pd.read_csv(r'C:\Users\msrg\Desktop\titanic\gender_submission.csv', header = 0, dtype={'Age': np.float64})
result_titanic.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


# Feature Engineering

In [4]:
# checking for missing values
full_data = [train_titanic,test_titanic]
for dataset in full_data:
    print(pd.isnull(dataset).sum()>0)
    break

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool


In [5]:
# checking the impact of the atrributes for the target attribute
# Pclass is categorical attribute
print(train_titanic[['Pclass','Survived']].groupby(['Pclass'], as_index=False).mean())

   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363


Pclass 1 is survived more compared to the other passenger classes

In [6]:
# Sex is a categorical variable
print(train_titanic[['Sex','Survived']].groupby(['Sex'], as_index=False).mean())

      Sex  Survived
0  female  0.742038
1    male  0.188908


Females are survived more comparec to the males

In [7]:
# SibSp is a numerical attribute which represent the number of spouse/sibilings
print(train_titanic[['SibSp','Survived']].groupby(['SibSp'],as_index=False).mean())

   SibSp  Survived
0      0  0.345395
1      1  0.535885
2      2  0.464286
3      3  0.250000
4      4  0.166667
5      5  0.000000
6      8  0.000000


In [8]:
# Parch is a numerical attribute
print(train_titanic[['Parch','Survived']].groupby(['Parch'],as_index=False).mean())

   Parch  Survived
0      0  0.343658
1      1  0.550847
2      2  0.500000
3      3  0.600000
4      4  0.000000
5      5  0.200000
6      6  0.000000


In [9]:
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp']+dataset['Parch']+1
print(train_titanic[['FamilySize','Survived']].groupby(['FamilySize'],as_index=False).mean())
    

   FamilySize  Survived
0           1  0.303538
1           2  0.552795
2           3  0.578431
3           4  0.724138
4           5  0.200000
5           6  0.136364
6           7  0.333333
7           8  0.000000
8          11  0.000000


In [10]:
train_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1


In [11]:
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
print(train_titanic[['IsAlone','Survived']].groupby(['IsAlone'],as_index=False).mean())

   IsAlone  Survived
0        0  0.505650
1        1  0.303538


In [12]:
train_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,1


In [13]:
# Embarked has missing values
train_titanic['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

Most occured value is S..so we can fill the missing values with the most occured value

In [14]:
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna(dataset['Embarked'].mode()[0])
print(train_titanic[['Embarked','Survived']].groupby(['Embarked'], as_index=False).mean())

  Embarked  Survived
0        C  0.553571
1        Q  0.389610
2        S  0.339009


Fare is a numerical atrribute. and has missing values. Also converted into a categorical attribute

In [15]:
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train_titanic['Fare'].median())
train_titanic['CatFare'] = pd.qcut(train_titanic['Fare'], 4)
print(train_titanic[['CatFare','Survived']].groupby(['CatFare'], as_index=False).mean())

           CatFare  Survived
0   (-0.001, 7.91]  0.197309
1   (7.91, 14.454]  0.303571
2   (14.454, 31.0]  0.454955
3  (31.0, 512.329]  0.581081


In [16]:
# age is a numerical atrribute. also has missing values. Because of that we need to generate random numbers.
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)

train_titanic['CatAge'] = pd.cut(train_titanic['Age'], 5)

print(train_titanic[['CatAge','Survived']].groupby(['CatAge'], as_index=False).mean())


          CatAge  Survived
0  (-0.08, 16.0]  0.551724
1   (16.0, 32.0]  0.357778
2   (32.0, 48.0]  0.351020
3   (48.0, 64.0]  0.434783
4   (64.0, 80.0]  0.090909


c:\python\python35\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
train_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone,CatFare,CatAge
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,2,0,"(-0.001, 7.91]","(16.0, 32.0]"
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,2,0,"(31.0, 512.329]","(32.0, 48.0]"
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,1,1,"(7.91, 14.454]","(16.0, 32.0]"
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,2,0,"(31.0, 512.329]","(32.0, 48.0]"
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,1,1,"(7.91, 14.454]","(32.0, 48.0]"


In [18]:

title_arr = []

for i in train_titanic['Name']:
    temp_array = i.split(" ")
    if "Mr." in temp_array:
        title_arr.append('Mr') 
    elif "Mrs." in temp_array:
        title_arr.append('Mrs')
    elif "Miss." in temp_array:
        title_arr.append('Miss')
    elif "Master." in temp_array:
        title_arr.append('Master')
    else:
        title_arr.append('Rare')
#print(title_arr)

train_titanic['Title'] = title_arr
train_titanic['Title'].value_counts()
#print (dataset[['Title','Survived']].groupby(['Title'], as_index=False).mean())
#print(dataset['Title'] )
#dataset.head()
#train_titanic.head()
print (train_titanic[['Title','Survived']].groupby(['Title'], as_index=False).mean())

    Title  Survived
0  Master  0.575000
1    Miss  0.697802
2      Mr  0.156673
3     Mrs  0.792000
4    Rare  0.444444


In [19]:
title_arr = []

for i in test_titanic['Name']:
    temp_arr = i.split(" ")
    if "Mr." in temp_arr:
        title_arr.append('Mr') 
    elif "Mrs." in temp_arr:
        title_arr.append('Mrs')
    elif "Miss." in temp_arr:
        title_arr.append('Miss')
    elif "Master." in temp_arr:
        title_arr.append('Master')
    else:
        title_arr.append('Rare')
#print(title_arr)

test_titanic['Title'] = title_arr
test_titanic['Title'].value_counts()
#print (dataset[['Title','Survived']].groupby(['Title'], as_index=False).mean())
#print(dataset['Title'] )
#dataset.head()
test_titanic.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone,Title
0,892,3,"Kelly, Mr. James",male,34,0,0,330911,7.8292,NaN,Q,1,1,Mr
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47,1,0,363272,7.0000,NaN,S,2,0,Mrs
2,894,2,"Myles, Mr. Thomas Francis",male,62,0,0,240276,9.6875,NaN,Q,1,1,Mr
3,895,3,"Wirz, Mr. Albert",male,27,0,0,315154,8.6625,NaN,S,1,1,Mr
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22,1,1,3101298,12.2875,NaN,S,3,0,Mrs


In [20]:
full_data = [train_titanic,test_titanic]

In [21]:
train_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone,CatFare,CatAge,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S,2,0,"(-0.001, 7.91]","(16.0, 32.0]",Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C,2,0,"(31.0, 512.329]","(32.0, 48.0]",Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S,1,1,"(7.91, 14.454]","(16.0, 32.0]",Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S,2,0,"(31.0, 512.329]","(32.0, 48.0]",Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S,1,1,"(7.91, 14.454]","(32.0, 48.0]",Mr


In [22]:
dataset.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,IsAlone,Title
0,892,3,"Kelly, Mr. James",male,34,0,0,330911,7.8292,NaN,Q,1,1,Mr
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47,1,0,363272,7.0000,NaN,S,2,0,Mrs
2,894,2,"Myles, Mr. Thomas Francis",male,62,0,0,240276,9.6875,NaN,Q,1,1,Mr
3,895,3,"Wirz, Mr. Albert",male,27,0,0,315154,8.6625,NaN,S,1,1,Mr
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22,1,1,3101298,12.2875,NaN,S,3,0,Mrs


In [23]:
for dataset in full_data:
    dataset['Sex'] = dataset['Sex'].fillna(0)
    dataset['Sex'] = dataset['Sex'].map({'female':0, 'male':1})
    
    title_mapping = {'Mr':1, 'Miss':2, 'Mrs':3, 'Master':4, 'Rare':5}
    dataset['Title'] = dataset['Title'].fillna(0)
    dataset['Title'] = dataset['Title'].map(title_mapping)
 
    
    dataset['Embarked'] = dataset['Embarked'].fillna(0)
    dataset['Embarked'] = dataset['Embarked'].map({'S':0,'C':1,'Q':2})
    
    dataset.loc[dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31.0), 'Fare'] = 2
    dataset.loc[(dataset['Fare'] > 31.0) & (dataset['Fare'] <= 512.329), 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(float)
    
    dataset.loc[dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[dataset['Age'] > 64, 'Age'] = 4
    dataset['Age'] = dataset['Age'].astype(float)
    
drop_elements = ['PassengerId','Name','Ticket','Cabin','SibSp','Parch','FamilySize']
train_titanic = train_titanic.drop(drop_elements, axis = 1)
train_titanic = train_titanic.drop(['CatAge','CatFare'], axis = 1)
test_passengers = pd.DataFrame(test_titanic['PassengerId'])
test_titanic = test_titanic.drop(drop_elements, axis = 1)

print(train_titanic.head())
     

   Survived  Pclass  Sex  Age  Fare  Embarked  IsAlone  Title
0         0       3    1  1.0   0.0         0        0      1
1         1       1    0  2.0   3.0         1        0      3
2         1       3    0  1.0   1.0         0        1      2
3         1       1    0  2.0   3.0         0        0      3
4         0       3    1  2.0   1.0         0        1      1


# K-Nearest Neighbors - 0.77990

In [24]:
from sklearn.neighbors import KNeighborsClassifier
#import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

train_labels = pd.DataFrame(train_titanic['Survived'])
train_features = train_titanic.drop(['Survived'], axis = 1)


In [25]:
train_features.head()

,Pclass,Sex,Age,Fare,Embarked,IsAlone,Title
0,3,1,1.0,0.0,0,0,1
1,1,0,2.0,3.0,1,0,3
2,3,0,1.0,1.0,0,1,2
3,1,0,2.0,3.0,0,0,3
4,3,1,2.0,1.0,0,1,1


In [26]:
X_train, X_test, y_train, y_test = train_test_split(train_features,train_labels,train_size=0.90)

c:\python\python35\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [27]:
knn=KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
train_prediction = knn.predict(X_test)

c:\python\python35\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


In [28]:
acc = accuracy_score(y_test,train_prediction)
print(acc)

0.8


In [29]:
knn=KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
result = knn.predict(test_titanic)
print(result)


[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 0 1 0 1 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0
 0 0 1 0 0 0 1 1 0 1 0 1 1 0 1 1 1 1 0 0 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 0 1
 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 1 0 1 0 0 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 1 0 0 1 0 0 0 1 1 0 0 0 1 1 1 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 1 0 1 0 0 1 0 0 0 1 1 1 1 1 0 0 1 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 1 0 1 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0
 1 0 1 0 0 0 1 0 0 0 1 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0
 0 1 0 1 1 1 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 1 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 1]


c:\python\python35\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


In [30]:
test_titanic.head()

,Pclass,Sex,Age,Fare,Embarked,IsAlone,Title
0,3,1,2.0,0.0,2,1,1
1,3,0,2.0,0.0,0,0,3
2,2,1,3.0,1.0,2,1,1
3,3,1,1.0,1.0,0,1,1
4,3,0,1.0,1.0,0,0,3


In [45]:
submission_file = test_titanic
submission_file['PassengerId'] = test_passengers
submission_file['Survived'] = result
drop_el = ['Pclass','Sex','Age','Fare','Embarked','IsAlone','Title']
submission_file = submission_file.drop(drop_el, axis=1)

In [46]:
submission_file.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [47]:
submission_file = pd.DataFrame(submission_file)

In [49]:
submission_file = submission_file.astype({"PassengerId": str})

In [52]:
submission_file.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [56]:
submission_file.to_csv("submission_file.csv", index = False)